In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
from music21 import *
from copy import deepcopy

In [ ]:
# load the datasets and then split in to train/test data sets
# note that we don't need label sets, only input sets for bidirectional LSTM (labels == inputs)
# unidirectional label set is the same as the inputs but without the first time step
with open("pickles/short_sequences_duration.pickle", 'rb') as short_duration,\
     open("pickles/short_sequences_duration.pickle", 'rb') as short_pitch:
    short_seqs_duration_train, short_seqs_duration_test = train_test_split(pickle.load(short_duration), train_size=0.95)
    short_seqs_pitch_train, short_seqs_pitch_test = train_test_split(pickle.load(short_pitch), train_size=0.95)

with open("pickles/medium_sequences_duration.pickle", 'rb') as medium_duration,\
     open("pickles/medium_sequences_duration.pickle", 'rb') as medium_pitch:
    medium_seqs_duration_train, medium_seqs_duration_test = train_test_split(pickle.load(medium_duration), train_size=0.95)
    medium_seqs_pitch_train, medium_seqs_pitch_test = train_test_split(pickle.load(medium_pitch), train_size=0.95)
    
with open("pickles/long_sequences_duration.pickle", 'rb') as long_duration,\
     open("pickles/long_sequences_duration.pickle", 'rb') as long_pitch:
    long_seqs_duration_train, long_seqs_duration_test = train_test_split(pickle.load(long_duration), train_size=0.95)
    long_seqs_pitch_train, long_seqs_pitch_test = train_test_split(pickle.load(long_pitch), train_size=0.95)

In [ ]:
# retrieve the pitches and durations that were used to build the data set
# these will be used to convert the output one-hot vectors back to actual pitch/duration values
with open('pickles/durations.pickle', 'rb') as d, open('pickles/pitches.pickle', 'rb') as p:
    durations = pickle.load(d)
    pitches = pickle.load(p)

num_durations = len(durations)
num_pitches = len(pitches)

In [ ]:
# single layer Unidirectional or Bidirectional LSTM; will easily allow us to test various configurations
def getModel(num_features, bidirectional=True):
    model = Sequential()
    # only dif. betwn. bi. LSTM and uni. LSTM is the presence/absence of Bidirectional wrapper
    # hidden layer 1; 20  units; input (# timesteps, # features); return a sequence of each time step's outputs
    # input_shape first value None makes it variable (we don't have fixed length sequences)
    if bidirectional:
        model.add(Bidirectional(LSTM(20, input_shape=(None, num_features), return_sequences=True)))
    else:
        model.add(LSTM(20, input_shape=(None, num_features), return_sequences=True))
        
    # TimeDistributed is a wrapper allowing one output per time step; 
    # ...requires hidden layer to have return_sequences == True
    model.add(TimeDistributed(Dense(num_features, activation='softmax')))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_crossentropy'])
    return model

In [ ]:
# train LSTM
def trainModel(model, X, epochs=30, bidirectional=True):
    print(X.shape)
    Y = deepcopy(X)
    if not bidirectional:
        Y = Y[1:] # labels include all time steps but the first one in unidir. LSTM
    model.fit(X, Y, epochs=epochs, verbose=1, validation_split=0.2)

In [ ]:
duration_bidirectional = getModel(num_durations, bidirectional=True)
trainModel(duration_bidirectional, short_seqs_duration_train, bidirectional=True)
trainModel(duration_bidirectional, medium_seqs_duration_train, bidirectional=True)
trainModel(duration_bidirectional, long_seqs_duration_train, bidirectional=True)

In [ ]:
for seq in medium_seqs_duration_train:
    print(seq.shape)

In [ ]:
pitch_bidirectional = getModel(num_pitches, bidirectional=True)
trainModel(pitch_bidirectional, short_seqs_pitch_train, bidirectional=True)
trainModel(pitch_bidirectional, medium_seqs_pitch_train, bidirectional=True)
trainModel(pitch_bidirectional, long_seqs_pitch_train, bidirectional=True)

In [ ]:
# model.predict requires 3D vector, so this reshapes a 2D input to 3D so it can be fed through the network
def to_3D(sample):
    return sample.reshape(1, sample.shape[0], sample.shape[1])

In [ ]:
# converts output one-hot vectors to its respective quarter length value (based on durations seen in data set)
def duration_one_hot_to_quarter_length(prediction):
    new_durations = []
    for timestep in prediction:
        index = np.argmax(timestep)
        new_durations.append(durations[index])
    
    return new_durations

# converts output one-hot vectors to its respective MIDI pitch value (based on durations seen in data set)
def pitch_one_hot_to_MIDI(prediction):
    new_pitches = []
    for timestep in prediction:
        index = np.argmax(timestep)
        new_pitches.append(pitches[index])
    
    return new_pitches

In [ ]:
duration_pred = duration_bidirectional.predict(to_3D(medium_seqs_duration_test[0])).reshape(timesteps, duration_num_features)
composed_durations = duration_one_hot_to_quarter_length(duration_pred)

pitch_pred = pitch_bidirectional.predict(to_3D(medium_seqs_pitch_test[0])).reshape(timesteps, pitch_num_features)
composed_pitches = pitch_one_hot_to_MIDI(pitch_pred)

In [ ]:
composed_pairs = list(zip(composed_pitches, composed_durations))

composed_stream = stream.Stream()
for pair in composed_pairs:
    p = pitch.Pitch(midi=pair[0])
    d = duration.Duration(pair[1])
    n = note.Note()
    n.pitch = p
    n.duration = d
    composed_stream.append(n)

In [ ]:
composed_stream.show('midi')

In [ ]:
composed_stream.show()